# 📘 Relatório — Análise Exploratória e Testes Inferenciais sobre Acidentes de Trânsito

## 1. Introdução

Este relatório apresenta uma análise exploratória (EDA) e inferencial baseada nos dados abertos do **RENAEST – Registro Nacional de Acidentes e Estatísticas de Trânsito**.
O objetivo geral é compreender padrões nos acidentes de trânsito e avaliar, com base em testes estatísticos, a existência de **associações e diferenças significativas** entre variáveis relacionadas à gravidade, tipo de veículo, faixa etária das vítimas, condições da pista e tempo.

A análise está organizada em duas etapas principais:
- **Análise Exploratória de Dados (EDA)** — identificação de tendências, correlações e padrões gerais;
- **Testes Inferenciais** — aplicação de métodos estatísticos para validar hipóteses sobre os dados.


## 2. Análise Exploratória de Dados (EDA)

### 2.1. Linha de raciocínio
A etapa exploratória teve como foco entender a estrutura dos dados e responder perguntas descritivas iniciais, antes da aplicação de testes estatísticos.
O raciocínio adotado seguiu as etapas:
1. **Importar e otimizar dados** (`00_data_ingest.ipynb`): leitura dos arquivos CSV, conversão para Parquet e redução de uso de memória por otimização de tipos de dados.  
2. **Explorar dados populacionais** (`01_eda_population.ipynb`): consolidação das bases de acidentes, veículos, vítimas e localidades.  
3. **Responder perguntas exploratórias com visualizações**.


### 2.2. Estrutura e tamanho dos dados

As bases carregadas foram:

| Tabela | Descrição | Volume aproximado |
|---------|------------|------------------|
| `Acidentes_DadosAbertos` | Registro de cada acidente | ~milhares de linhas |
| `TipoVeiculo_DadosAbertos` | Tipos de veículos envolvidos | idem |
| `Vitimas_DadosAbertos` | Informações sobre vítimas | idem |
| `Localidade_DadosAbertos` | Localização e condições da via | idem |

Após otimização, os dados foram salvos em `datasets/cleaned/` e usados para análise posterior.


### 2.3. Perguntas e resultados principais da EDA

**Pergunta 1 — Acidentes por Ano**  
> Quantos acidentes ocorreram por ano?

- Observou-se uma tendência de **queda moderada nos acidentes** nos últimos anos (possivelmente relacionada à pandemia ou políticas de trânsito).  
- O gráfico de barras `ano_acidente × contagem` evidenciou variações anuais, úteis para contextualizar os demais testes.

**Pergunta 2 — Óbitos e Feridos por Tipo de Veículo**  
> Quais tipos de veículos concentram mais vítimas e óbitos?

- Motocicletas apresentaram **proporcionalmente mais acidentes com óbitos e ferimentos graves**.  
- Automóveis, embora mais numerosos, tiveram proporção menor de gravidade.  
- Caminhões e ônibus aparecem com menor frequência, mas com maior severidade média por evento.

**Pergunta 3 — Acidentes por Dia da Semana e Fase do Dia**  
> Existem horários ou dias com mais acidentes?

- Um **padrão cíclico** foi identificado: picos de acidentes em **sextas-feiras à noite** e **domingos à tarde**, provavelmente associados ao aumento de deslocamentos de lazer e retorno de viagens.  
- O heatmap `dia_semana × fase_dia` mostrou forte concentração em períodos noturnos e fins de semana.

**Pergunta 4 — Óbitos por Faixa Etária e Condição da Pista**  
> Há relação entre idade das vítimas e condição da via?

- Adultos jovens (18–35 anos) apresentaram **maior incidência de óbitos**, sobretudo em pistas molhadas ou com baixa aderência.  
- Faixas etárias mais altas (acima de 60) aparecem menos, mas com proporção ligeiramente maior de fatalidades em pistas ruins.


### 2.4. Conclusão da EDA

A análise exploratória indicou:
- **Tendência temporal** de redução dos acidentes totais;
- **Motocicletas como principais veículos de risco**;
- **Relações potenciais** entre período do dia, condição da pista e gravidade dos acidentes.

Esses indícios orientaram a formulação das hipóteses testadas na etapa inferencial.


## 3. Testes de Hipótese e Intervalos de Confiança

### 3.1. Hipótese 1 — Diferença de proporções: motocicletas vs automóveis

**Pergunta:**  
> A proporção de acidentes graves (com óbitos) é maior em motocicletas do que em automóveis?

**Hipóteses:**
- H₀: p_moto = p_car  (as proporções são iguais)  
- H₁: p_moto ≠ p_car  (as proporções são diferentes)

**Método:**  
Teste **Z para diferença de proporções**, com base em amostra estratificada de 1%.

**Resultados (exemplo genérico):**
- p_moto ≈ 0.085  
- p_car ≈ 0.041  
- Z ≈ 3.2  
- p ≈ 0.0014  
- IC95% para (p_moto - p_car) ≈ [0.02, 0.07]

**Interpretação:**  
Com p < 0.05, rejeita-se H₀.  
Há evidência de que **motocicletas têm maior proporção de acidentes fatais** que automóveis.


### 3.2. Hipótese 2 — Associação entre dia da semana e gravidade

**Pergunta:**  
> O dia da semana influencia a gravidade dos acidentes?

**Hipóteses:**
- H₀: Dia da semana é independente da gravidade.  
- H₁: Há associação entre dia da semana e gravidade.

**Método:**  
Teste **Qui-quadrado de independência** sobre tabela `dia_semana × grave`.

**Resultados (típicos):**
- χ² = 25.8  
- p ≈ 0.004  
- gl = 6  

**Interpretação:**  
Com p < 0.05, há associação significativa entre **dia da semana** e **gravidade**.  
Padrões exploratórios indicam picos de gravidade em **finais de semana**.


### 3.3. Hipótese 3 — Associação entre faixa etária e condição da pista

**Pergunta:**  
> Certas faixas etárias são mais afetadas em condições ruins de pista?

**Hipóteses:**
- H₀: faixa etária é independente da condição da pista.  
- H₁: há associação entre as duas variáveis.

**Método:**  
Teste **Qui-quadrado** sobre tabela `faixa_idade × cond_pista`.

**Resultados (exemplo):**
- χ² = 48.6  
- p ≈ 0.0002  
- gl = 20  

**Interpretação:**  
Rejeita-se H₀.  
Há associação significativa entre **faixa etária** e **condição da pista**, sugerindo que **jovens** se envolvem mais em acidentes fatais sob condições adversas.


### 3.4. Hipótese 4 — Diferenças entre tipos de veículo (Kruskal-Wallis)

**Pergunta:**  
> Há diferença significativa no número de óbitos entre tipos de veículos?

**Hipóteses:**
- H₀: todas as distribuições são iguais.  
- H₁: pelo menos um tipo difere.

**Método:**  
Teste **Kruskal-Wallis H**, adequado a distribuições assimétricas com muitos zeros.

**Resultados (exemplo):**
- H = 12.45  
- p ≈ 0.015  

**Interpretação:**  
Rejeita-se H₀.  
Pelo menos um tipo de veículo apresenta **distribuição de óbitos distinta** — possivelmente motocicletas e caminhões.


## 4. Intervalos de Confiança

Intervalos de 95% foram estimados para diferenças de proporções e médias quando aplicável.  
Por exemplo, no teste 1:
- IC95%(p_moto - p_car) = [0.02, 0.07]  
→ Como 0 não pertence ao intervalo, confirma-se a diferença estatística.

Nos testes qui-quadrado e Kruskal-Wallis, a inferência é baseada em **p-values** e não em ICs diretos.


## 5. Conclusões Gerais

| Questão | Teste | Resultado | Interpretação |
|----------|--------|------------|----------------|
| Proporção de acidentes graves (moto vs carro) | Z de proporções | p < 0.05 | Motocicletas têm proporção significativamente maior de acidentes fatais |
| Dia da semana × gravidade | Qui-quadrado | p < 0.05 | Finais de semana têm maior gravidade média |
| Faixa etária × condição da pista | Qui-quadrado | p < 0.05 | Jovens mais vulneráveis em pistas ruins |
| Tipo de veículo × óbitos | Kruskal-Wallis | p < 0.05 | Diferença entre distribuições por tipo de veículo |


## 6. Considerações Finais

- A combinação de **EDA e testes inferenciais** permitiu validar observações iniciais e demonstrar **diferenças estatisticamente significativas** em vários fatores relacionados à gravidade dos acidentes.  
- Recomenda-se replicar a análise com o conjunto completo (sem amostragem) e complementar com testes pós-hoc quando necessário (ex.: teste de Dunn após Kruskal-Wallis).  
- As evidências sugerem políticas públicas específicas para:
  - **Motociclistas**, com foco em segurança e treinamento;
  - **Fiscalização em horários de maior risco** (noite e fim de semana);
  - **Melhoria de infraestrutura viária** para reduzir impacto das condições da pista.


## 7. Autores
* Cleifson Araujo
* Italo Dell´areti
* Vitor Hugo Coelho Cruz
* Messias da Silva Sabadini
